In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    temperature=0.1
)
# temperature: 얼마나 창의적인가 0~1

template = PromptTemplate.from_template(
    "{country_a}과(와) {country_b} 사이의 거리는 어떻게 되나요?"
)

prompt = template.format(country_a = "한국", country_b= "일본")


chat.predict(prompt)

# from langchain.llms import OpenAI
# llm = OpenAI(model_name="gpt-3.5-turbo-0125")
# a = llm.predict("How many planets are there?")
# a

'한국과 일본 사이의 거리는 직선거리로 약 1,000km 정도입니다. 그러나 실제로 이동할 때는 해상이기 때문에 비행기나 배 등을 이용하여 이동해야 합니다. 한국과 일본 사이에는 인천과 도쿄를 연결하는 항공편이 운행되고 있으며, 또한 여러 해운사들이 한국과 일본 간의 해상 운송을 제공하고 있습니다. 따라서 이동 수단에 따라 거리는 달라질 수 있지만, 대체로 한국과 일본 사이의 이동 거리는 약 1,000km 정도로 생각할 수 있습니다.'

In [15]:
# from langchain.schema import HumanMessage, AIMessage, SystemMessage

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography export. And you only reply in {language}."),
    ("ai", "안녕하세요. 제 이름은 {name}입니다."),
    ("human", "{country_a}과(와) {country_b} 사이의 거리는 어떻게 되나요?. 그리고 당신의 이름은 무엇인가요?"),
])

prompt = template.format_messages(
    language = "korean",
    name = "쫀떡",
    country_a = "한국",
    country_b = "일본",
)

chat.predict_messages(prompt)

AIMessage(content='한국과 일본 사이의 거리는 약 1,000km 정도입니다. 제 이름은 쫀떡입니다.')

In [17]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    # strip: text 앞뒤 공백을 잘라준다. -> 텍스트ㅡ이 시작부분이나 끝부분이 공백이라면 제거
    
p = CommaOutputParser()
p.parse("Hello,   how,are,you")

['Hello', 'how', 'are', 'you']

In [27]:
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 list 생성 기계입니다. 질문들은 모두 최대 {max_items}개, comma로 구분해주세요. 그리고 영어로 답할 경우 소문자로만 답해주세요. list가 아닌 것으로 답을 하지마세요"),
    ("human", "{question}"),
    ])

prompt = template.format_messages(max_items=  "10", question = "What are the colors?")
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import BaseOutputParser

chat = ChatOpenAI(
    temperature=0.1
)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    # strip: text 앞뒤 공백을 잘라준다. -> 텍스트ㅡ이 시작부분이나 끝부분이 공백이라면 제거
    
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 list 생성 기계입니다. 질문들은 모두 최대 {max_items}개, comma로 구분해주세요. 그리고 영어로 답할 경우 소문자로만 답해주세요. list가 아닌 것으로 답을 하지마세요"),
    ("human", "{question}"),
    ])

prompt = template.format_messages(max_items=  "10", question = "What are the colors?")
result = chat.predict_messages(prompt)
p = CommaOutputParser()
p.parse(result.content)

# chat model, outputParser, template

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [4]:
# chain: 모든 요소를 합쳐주는 역할을 한다.
# 합쳐진 요소들은 하나의 chain으로 실행된다. 

chain_one = template | chat | CommaOutputParser()

# chain_two = template_2 | chat | outputparser

# all = chain_one | chain_two | outputparser 

chain_one.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']